# Hyperparamètre

In [1]:
train_keras = True
train_qkeras =True
model_name = 'AlexNet_Weights/AlexNet_Cifar10_Keras.h5'
qmodel_name_8 = 'AlexNet_Weights/AlexNet_Cifar10_QKeras_8.h5'
qmodel_name_16 = 'AlexNet_Weights/AlexNet_Cifar10_QKeras_16.h5'
epochs = 300
batch_size = 1024
lr = 0.001
log_dir_keras = 'AlexNet_Cifar10_LOG/Keras'
log_dir_qkeras_8 = 'AlexNet_Cifar10_LOG/QKeras_8'
log_dir_qkeras_16 = 'AlexNet_Cifar10_LOG/QKeras_16'

# Dataset Part

In [2]:
import tensorflow as tf
import keras
import time
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

2023-06-13 17:38:30.832734: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
devices = tf.config.list_physical_devices()

for device in devices:
    print(device)

PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2023-06-13 17:38:32.394631: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-13 17:38:32.466306: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-13 17:38:32.466661: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.74GiB deviceMemoryBandwidth: 573.69GiB/s
2023-06-13 17:38:32.466679: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-13 17:38:32.475729: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-13 17:38:32.475769: I tensorflow/stream_executor

In [4]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images = train_images.astype("float32") / 255
test_images = test_images.astype("float32") / 255

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Keras Part

In [5]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Model

filters_per_conv_layer = [64, 4, 8, 16, 16]
neurons_per_dense_layer = [16, 10]

x = x_in = Input((32, 32, 3)) 

n_classes = 10 

for i, f in enumerate(filters_per_conv_layer):
    print(('Adding convolutional block {} with N={} filters').format(i, f))
    x = Conv2D(
        int(f),
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.001),
        use_bias=False,
        name='conv_{}'.format(i),
    )(x)
    x = BatchNormalization(name='bn_conv_{}'.format(i))(x)
    x = Activation('relu', name='conv_act_%i' % i)(x)
    if i == 0 or i == 1 or i == 4:  # Apply pooling after layers 1, 2 and 5
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool_{}'.format(i))(x)
x = Flatten()(x)

for i, n in enumerate(neurons_per_dense_layer):
    print(('Adding dense block {} with N={} neurons').format(i, n))
    x = Dense(n, kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.001), name='dense_%i' % i, use_bias=False)(x)
    x = BatchNormalization(name='bn_dense_{}'.format(i))(x)
    x = Activation('relu', name='dense_act_%i' % i)(x)

x = Dense(int(n_classes), name='output_dense')(x)
x_out = Activation('softmax', name='output_softmax')(x)

keras_model = Model(inputs=[x_in], outputs=[x_out], name='alexnet')

keras_model.summary()


Adding convolutional block 0 with N=64 filters


2023-06-13 17:38:33.142769: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 17:38:33.143790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-13 17:38:33.145589: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.74GiB deviceMemoryBandwidth: 573.69GiB/s
2023-06-13 17:38:33.145681: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (

Adding convolutional block 1 with N=4 filters
Adding convolutional block 2 with N=8 filters
Adding convolutional block 3 with N=16 filters
Adding convolutional block 4 with N=16 filters
Adding dense block 0 with N=16 neurons
Adding dense block 1 with N=10 neurons
Model: "alexnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv_0 (Conv2D)              (None, 32, 32, 64)        1728      
_________________________________________________________________
bn_conv_0 (BatchNormalizatio (None, 32, 32, 64)        256       
_________________________________________________________________
conv_act_0 (Activation)      (None, 32, 32, 64)        0         
_________________________________________________________________
pool_0 (MaxPooling2D)        (None, 16, 16, 64)        0   

# Train

In [6]:
datagen = ImageDataGenerator(
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def step_decay(epoch):
    initial_lr = lr
    drop = 0.5
    epochs_drop = 10
    lr = initial_lr * drop * (epoch // epochs_drop)
    return lr

#train_datagen = datagen.flow(train_images, train_labels, batch_size=batch_size)

keras_model.compile(
    optimizer=Adam(lr),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir_keras,
    histogram_freq=1
)
start = time.time()
keras_model.fit(
    #datagen.flow(train_images, train_labels, batch_size=batch_size),
    train_images,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(test_images, test_labels),
    verbose=1,
    callbacks=[tensorboard_callback]
)
end = time.time()

keras_model.save(model_name)
print('It took {} minutes to train Keras model'.format((end - start) / 60.0))

2023-06-13 17:38:33.832356: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-06-13 17:38:33.832382: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-06-13 17:38:33.832415: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2023-06-13 17:38:33.832704: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64::/usr/local/hdf5/lib
2023-06-13 17:38:33.832784: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64::/usr/local/hdf5/lib
2023-06-13 17:38:33.832796: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1

Epoch 1/300


2023-06-13 17:38:35.959133: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-06-13 17:38:36.433795: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2023-06-13 17:38:37.365935: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-13 17:38:37.956641: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


 8/49 [===>..........................] - ETA: 1s - loss: 3.3493 - accuracy: 0.1217

2023-06-13 17:38:38.605415: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-06-13 17:38:38.605438: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-06-13 17:38:38.605465: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2023-06-13 17:38:38.628333: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2023-06-13 17:38:38.628402: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2023-06-13 17:38:38.631164: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2023-06-13 17:38:38.632433: I tensorflow

...

49/49 [==============================] - 1s 23ms/step - loss: 1.0793 - accuracy: 0.7171 - val_loss: 1.4458 - val_accuracy: 0.6025
Epoch 59/300
49/49 [==============================] - 1s 23ms/step - loss: 1.0760 - accuracy: 0.7197 - val_loss: 1.3790 - val_accuracy: 0.6274
Epoch 60/300
49/49 [==============================] - 1s 23ms/step - loss: 1.0752 - accuracy: 0.7177 - val_loss: 1.4523 - val_accuracy: 0.5977
Epoch 61/300
49/49 [==============================] - 1s 23ms/step - loss: 1.0733 - accuracy: 0.7185 - val_loss: 1.3961 - val_accuracy: 0.6225
Epoch 62/300
49/49 [==============================] - 1s 23ms/step - loss: 1.0673 - accuracy: 0.7226 - val_loss: 1.3384 - val_accuracy: 0.6370
Epoch 63/300
49/49 [==============================] - 1s 23ms/step - loss: 1.0680 - accuracy: 0.7207 - val_loss: 1.4308 - val_accuracy: 0.6061
Epoch 64/300
49/49 [==============================] - 1s 23ms/step - loss: 1.0645 - accuracy: 0.7207 - val_loss: 1.4315 - val_accuracy: 0.6163
Epoch 65/300

49/49 [==============================] - 1s 23ms/step - loss: 0.9803 - accuracy: 0.7434 - val_loss: 1.3903 - val_accuracy: 0.6245
Epoch 116/300
49/49 [==============================] - 1s 23ms/step - loss: 0.9779 - accuracy: 0.7442 - val_loss: 1.2943 - val_accuracy: 0.6451
Epoch 117/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9711 - accuracy: 0.7484 - val_loss: 1.4527 - val_accuracy: 0.6134
Epoch 118/300
49/49 [==============================] - 1s 23ms/step - loss: 0.9770 - accuracy: 0.7423 - val_loss: 1.3200 - val_accuracy: 0.6438
Epoch 119/300
49/49 [==============================] - 1s 23ms/step - loss: 0.9828 - accuracy: 0.7408 - val_loss: 1.4953 - val_accuracy: 0.6202
Epoch 120/300
49/49 [==============================] - 1s 23ms/step - loss: 0.9778 - accuracy: 0.7431 - val_loss: 1.3639 - val_accuracy: 0.6336
Epoch 121/300
49/49 [==============================] - 1s 23ms/step - loss: 0.9764 - accuracy: 0.7448 - val_loss: 1.4265 - val_accuracy: 0.6186
Epoch 

Epoch 172/300
49/49 [==============================] - 1s 23ms/step - loss: 0.9370 - accuracy: 0.7545 - val_loss: 1.3543 - val_accuracy: 0.6353
Epoch 173/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9370 - accuracy: 0.7536 - val_loss: 1.3052 - val_accuracy: 0.6475
Epoch 174/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9425 - accuracy: 0.7525 - val_loss: 1.3014 - val_accuracy: 0.6516
Epoch 175/300
49/49 [==============================] - 1s 23ms/step - loss: 0.9388 - accuracy: 0.7523 - val_loss: 1.3329 - val_accuracy: 0.6431
Epoch 176/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9305 - accuracy: 0.7563 - val_loss: 1.4310 - val_accuracy: 0.6243
Epoch 177/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9386 - accuracy: 0.7534 - val_loss: 1.4929 - val_accuracy: 0.6093
Epoch 178/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9309 - accuracy: 0.7560 - val_loss: 1.3106 - val_accuracy:

Epoch 229/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9139 - accuracy: 0.7588 - val_loss: 1.3083 - val_accuracy: 0.6491
Epoch 230/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9121 - accuracy: 0.7602 - val_loss: 1.4503 - val_accuracy: 0.6118
Epoch 231/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9117 - accuracy: 0.7619 - val_loss: 1.2571 - val_accuracy: 0.6595
Epoch 232/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9086 - accuracy: 0.7625 - val_loss: 1.4665 - val_accuracy: 0.6073
Epoch 233/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9204 - accuracy: 0.7576 - val_loss: 1.3024 - val_accuracy: 0.6516
Epoch 234/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9197 - accuracy: 0.7572 - val_loss: 1.2771 - val_accuracy: 0.6550
Epoch 235/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9215 - accuracy: 0.7570 - val_loss: 1.3529 - val_accuracy:

Epoch 286/300
49/49 [==============================] - 1s 24ms/step - loss: 0.8971 - accuracy: 0.7635 - val_loss: 1.2972 - val_accuracy: 0.6453
Epoch 287/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9004 - accuracy: 0.7627 - val_loss: 1.5360 - val_accuracy: 0.5983
Epoch 288/300
49/49 [==============================] - 1s 24ms/step - loss: 0.8922 - accuracy: 0.7669 - val_loss: 1.2944 - val_accuracy: 0.6574
Epoch 289/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9007 - accuracy: 0.7644 - val_loss: 1.3470 - val_accuracy: 0.6306
Epoch 290/300
49/49 [==============================] - 1s 24ms/step - loss: 0.8943 - accuracy: 0.7667 - val_loss: 1.4888 - val_accuracy: 0.6085
Epoch 291/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9019 - accuracy: 0.7619 - val_loss: 1.3137 - val_accuracy: 0.6531
Epoch 292/300
49/49 [==============================] - 1s 24ms/step - loss: 0.9001 - accuracy: 0.7648 - val_loss: 1.2600 - val_accuracy: